▶️ **Link to Youtube Video:** [Day 9 - Working with raster files in Python | Introduction to Rasterio (Part 3)](https://youtu.be/yp8mzWAut58?si=a63SjMJJpNLBZSP1)


▶️ **Link to Full Youtube Playlist:** [12 Days Geospatial Python Bootcamp](https://youtube.com/playlist?list=PLPBWT_CJ5QhL90iN3n6zWGpSXQLw42ToU&si=04Dv0mI3pPpBK29z)

## Working with Rasters in Python
### Introduction to Rasterio (Part 3)

In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import os

### Compositing / Stacking

In [ ]:
images_dir = os.path.normpath("./rasters") # the path to the folder which contains your images

bands = [file for file in os.listdir(images_dir) if file.endswith(".tif")]
sorted_bands = sorted(bands)

# Empty list to store all the files
stack_list = []

for band in sorted_bands:
    file = os.path.join(images_dir, band)

    # Open the image
    with rio.open(file) as src:
        # Read band into numpy array and append to stack list
        raster_band = src.read(1)
        stack_list.append(raster_band)

# Stack all bands
stacked_bands = np.stack(stack_list, axis=0)

In [ ]:
red = stack_list[5]
blue = stack_list[4]
green = stack_list[3]

rgb = np.dstack((red, green, blue))

rgb_normalized = (rgb - np.min(rgb)) / (np.max(rgb) - np.min(rgb))

plt.figure(figsize=(8, 8))
plt.imshow(rgb_normalized)
plt.show()

### Saving Composite Raster

In [ ]:
band_1_path = r"path to an existing satellite image band"

with rio.open(band_1_path) as dataset:
    meta = dataset.meta
    print("Dataset shape:", dataset.read(1).shape)

meta.update({
    "count": 3
})

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

output_raster = os.path.join(output_dir, "composite.tif")

# Transpose the stacked array
output_data = np.transpose(rgb_normalized, (2, 0, 1))
print("Composite shape:", output_data.shape)
print("Normalized shape:", rgb_normalized.shape)

with rio.open(output_raster, "w", **meta) as dst:
    dst.write(output_data)


In [ ]:
with rio.open(output_raster) as file:
    print(file.count)

## Reprojecting a raster

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
import pyproj

composite_raster = rio.open('./output/composite.tif')
composite_raster.crs

crs = "EPSG:4326"
dst_crs = pyproj.CRS(crs)

In [ ]:
# Open the original image
with rio.open('./output/composite.tif') as src:
    # band = src.read(1)
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    metadata = src.meta.copy()
    metadata.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Reporoject the raster
    output_reprojected = os.path.join(output_dir, "reprojected.tif")
    with rio.open(output_reprojected, 'w', **metadata) as dst:
        for i in range(1, src.count + 1):
            reproject(
            source=rio.band(src, i),
            destination=rio.band(dst, i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )




In [ ]:
projected = rio.open(output_reprojected)
projected.crs

### Cropping a raster with a shapefile

In [ ]:
import geopandas as gpd
from rasterio.mask import mask as rmask


roi = r"/home/tommy/12_Days_GeoPython_Training/Day_9/shps/roi.shp"
composite_path = r"/home/tommy/12_Days_GeoPython_Training/Day_9/output/composite.tif"
# composite_path = r"/home/tommy/12_Days_GeoPython_Training/Day_9/rasters/Band_1.tif"

clipped_path = os.path.join(output_dir, "clipped.tif")

gdf = gpd.read_file(roi)


with rio.open(composite_path) as src:
    shapes = gdf.geometry

    cropped_image, cropped_image_transform = rmask(src, shapes, crop=True)

    out_meta = src.meta.copy()
    out_meta.update({
        "height": cropped_image.shape[1],
        "width": cropped_image.shape[2],
        "transform": cropped_image_transform
    })

    # Save the cropped file
    with rio.open(clipped_path, "w", **out_meta) as dst:
        dst.write(cropped_image)
        # dst.write(cropped_image[i - 1], i)